python3 -m build# Pre-processing of LAS tiles

Pre-processing consists of two parts:
1. Clipping of AHN point clouds to match target point clouds;
2. Ground and building surface extraction.

In [ ]:
# Uncomment to load the local package rather than the pip-installed version.
# Add project src to path.
import set_path

In [ ]:
# Import modules.
import numpy as np
import time

import upcp.preprocessing.ahn_preprocessing as ahn_preprocessing
import upcp.utils.ahn_utils as ahn_utils
import upcp.utils.las_utils as las_utils
import upcp.utils.plot_utils as plot_utils

In [ ]:
# Set-up.

# The point cloud tile for which a corresponding AHN tile should be created.
tile_code = '2386_9702'
filename = '../datasets/pointcloud/filtered_' + tile_code + '.laz'

# The folder where AHN data will be saved.
ahn_data_folder = '../datasets/ahn/'

## 1a. Clip a single file

In [ ]:
# Download the corresponding AHN cloud from 
# https://www.arcgis.com/apps/Embed/index.html?appid=a3dfa5a818174aa787392e461c80f781
!wget https://download.pdok.nl/rws/ahn3/v1_0/laz/C_25DN2.LAZ -P ../datasets/

In [ ]:
# Load the AHN point cloud.
# WARNING: this may require 16+ GB of RAM, depending on the size of the cloud.
ahn_cloud_file = '../datasets/C_25DN2.LAZ'
ahn_cloud = las_utils.read_las(ahn_cloud_file)

In [ ]:
# Clip the corresponding AHN point cloud tile.
start = time.time()
ahn_preprocessing.clip_ahn_las_tile(ahn_cloud, filename, out_folder=ahn_data_folder, buffer=1)
end = time.time()
print(f'Tile clipped in {end-start:.2f} seconds.')

## 1b. Clip all files in a folder

In [ ]:
las_folder = '../datasets/pointcloud/'

# Set resume=True to skip existing files in the out_folder.
ahn_preprocessing.clip_ahn_las_folder(ahn_cloud, las_folder, out_folder=ahn_data_folder, buffer=1, resume=False)

## 2a. Surface extraction for a single file

In [ ]:
# Load the clipped tile that we have just created.
ahn_las_file = ahn_data_folder + '/ahn_' + tile_code + '.laz'

start = time.time()
# This will create the AHN ground and building surfaces, with a 0.1m resolution, and save them as an .npz file.
_file = ahn_preprocessing.process_ahn_las_tile(ahn_las_file, out_folder=ahn_data_folder)
end = time.time()
print(f'Processed LAS tile in {end-start:.2f}s.')

In [ ]:
# Load the result.
ahn_reader = ahn_utils.NPZReader(ahn_data_folder)

In [ ]:
# Optional: plot the resulting surfaces.
%matplotlib widget
plot_utils.plot_ahn_sidebyside(tile_code, ahn_reader)

This will look like:

![Surface Plot](https://github.com/Amsterdam-AI-Team/Urban_PointCloud_Processing/raw/main/media/examples/surface_plot.png)

## 2b. Surface extraction for a folder of ahn cloud tiles

We have a batch script to do this, it allows using multiple CPU threads to speed things up.

```sh
$ cd ../scripts
$ python ahn_batch_processor.py --in_folder ../datasets/ahn/ --out_folder ../datasets/ahn/ --resume --workers 4
```